In [37]:
import numpy as np
from numpy.typing import NDArray

'''The jacobian logarithm, used in log likelihood normalization and resampling processes
δ will be an array of log-probabilities '''
def jacob(δ:NDArray)->NDArray:
    '''δ is our vector of weights'''
    n = len(δ)
    Δ = np.zeros(n) #Δ is a vector of running sums, i.e. the log-cdf
    Δ[0] = δ[0]
    for i in range(1,n):
        Δ[i] = max(δ[i],Δ[i-1]) + np.log(1 + np.exp(-1*np.abs(δ[i] - Δ[i-1]))) 
    return(Δ)

'''normalizes the probability space using the jacobian logarithm as defined in jacob() '''
def log_norm(log_weights:NDArray): 
    norm = (jacob(log_weights)[-1])
    log_weights -= norm
    return log_weights


gen = np.random.default_rng(5)
weights = np.array([gen.uniform() for _ in range(10)])
weights /= np.sum(weights)

'''The default numpy implementation of the resampling complexity O(NlogN), the binary search is performed N times'''
cdf = weights.cumsum()
uinform_samples = gen.random(weights.shape) #Draws N random samples on the interval U[0,1)
indices = cdf.searchsorted(uinform_samples,side='right') #performs a binary search to find where the samples would be in order inserted on the right
print(f"Numpy binary resample indices: {indices}")



N = len(weights) #number of weights

'''Systematic resampling in lin-domain, complexity O(N), only one sample is drawn, offsets from that sample are constant'''
i=0
indices = np.zeros(N)
u = np.zeros(N)
u[0] = gen.uniform(0,1/N)
for j in range(0,N):
    u[j] = u[0] + 1/N*(j)
    while u[j] > cdf[i]:
        i+=1
    indices[j]=i
print(f"SR indices:{indices}")

'''Systematic resampling in the log-domain requires special function, the Jacobian Logarithm'''

'''we must first normalize the weights in the log-domain, this can be done using func:jacob'''
log_weights = np.log(weights)
log_weights = log_norm(log_weights)

indices = np.zeros(N)
log_cdf = jacob(log_weights)
i = 0
u = gen.uniform(0,1/N)
for j in range(0,N):
    r = np.log(u + 1/N*(j))
    while r>log_cdf[i]:
        i = i + 1
    indices[j] = i
print(f"Log-SR indices:{indices}")
    



Numpy binary resample indices: [5 1 2 9 9 9 2 3 6 0]
SR indices:[0. 0. 1. 1. 2. 4. 6. 7. 9. 9.]
0.9999999999999999
Log-SR indices:[0. 0. 1. 1. 2. 3. 5. 6. 9. 9.]
